In [4]:
# ================= CONFIGURATION =================
# Set your school's semester dates here
SEMESTER_START = "2026-03-15"  # YYYY-MM-DD
SEMESTER_END   = "2026-07-03"  # YYYY-MM-DD

# Set the working directory for the script (where it will read/write files)
WORKING_DIRECTORY = "/Users/roykisluk/Downloads/"
# =================================================

In [5]:
# set working directory
import os
os.chdir(WORKING_DIRECTORY)

In [ ]:
import json
import uuid
from datetime import datetime, timedelta

def get_first_occurrence(start_date_str, target_day):
    days = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, 
            "Friday": 4, "Saturday": 5, "Sunday": 6}
    current_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    target_weekday = days[target_day]
    while current_date.weekday() != target_weekday:
        current_date += timedelta(days=1)
    return current_date.strftime("%Y%m%d")

def create_ics(json_file, output_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        events = json.load(f)

    until_date = SEMESTER_END.replace("-", "")
    ics_content = [
        "BEGIN:VCALENDAR",
        "VERSION:2.0",
        "PRODID:-//UniSchedule//EN",
        "CALSCALE:GREGORIAN",
        "METHOD:PUBLISH",
        "X-WR-TIMEZONE:Asia/Jerusalem"  # Added specific timezone hint
    ]

    for e in events:
        timestamp = datetime.now().strftime('%Y%m%dT%H%M%SZ')
        location = e.get('location', 'Berglas 101')
        summary = e['summary']
        
        # Unique ID for every single entry to prevent "disappearing" events
        event_uid = str(uuid.uuid4()) 

        if e.get('date') == "weekly":
            first_date = get_first_occurrence(SEMESTER_START, e['day'])
            start_dt = f"{first_date}T{e['start'].replace(':', '')}00"
            end_dt = f"{first_date}T{e['end'].replace(':', '')}00"
            day_map = {"Monday": "MO", "Tuesday": "TU", "Wednesday": "WE", "Thursday": "TH", "Friday": "FR"}
            rrule = f"RRULE:FREQ=WEEKLY;BYDAY={day_map[e['day']]};UNTIL={until_date}T235959Z"
        else:
            # FIXED DATE LOGIC
            date_clean = e['date'].replace('-', '').replace('.', '')
            start_dt = f"{date_clean}T{e['start'].replace(':', '')}00"
            end_dt = f"{date_clean}T{e['end'].replace(':', '')}00"
            rrule = None

        ics_content.append("BEGIN:VEVENT")
        ics_content.append(f"UID:{event_uid}") # Uses a truly unique ID
        ics_content.append(f"SUMMARY:{summary}")
        ics_content.append(f"LOCATION:{location}")
        ics_content.append(f"DTSTART:{start_dt}")
        ics_content.append(f"DTEND:{end_dt}")
        ics_content.append(f"DTSTAMP:{timestamp}")
        if rrule:
            ics_content.append(rrule)
        ics_content.append("END:VEVENT")

    ics_content.append("END:VCALENDAR")

    with open(output_file, "w", encoding="utf-8") as file:
        file.write("\n".join(ics_content))

    print(f"File '{output_file}' generated successfully.")

create_ics("course_data.json", "school_schedule_v2.ics")

File 'school_schedule_v2.ics' generated. Try re-importing this version.
